Import required libraries

In [1]:
import subprocess   # To run external commands (e.g., QGIS processing model)
import requests     # To make HTTP requests (e.g., upload image to imgbb)
import os           # For file system operations (e.g., deleting the image file)

Image upload function

In [2]:
# 🔐 Your imgbb API key
CLIENT_ID = 'f100258a2d21b41e27b26c8b6a474f98'

# 📤 Function to upload an image to imgbb.com
def upload_to_imgbb(image_path, title=None):
    url = 'https://api.imgbb.com/1/upload'  # imgbb upload endpoint

    # Open the image file in binary read mode
    with open(image_path, 'rb') as img:
        data = {
            'key': CLIENT_ID,       # API key for authentication
            'expiration': 0,        # 0 = image never expires
            'name': title or None   # Optional name for the image
        }
        files = {
            'image': img            # Binary image data
        }

        # Send POST request to imgbb
        r = requests.post(url, data=data, files=files)

        # Raise an exception if upload fails
        r.raise_for_status()

        # Extract and return the URL of the uploaded image
        result = r.json()['data']
        print('✅ Uploaded:', result['url'])
        return result['url']

QGIS command

In [3]:
# 🗺️ Step 1: Run QGIS model using qgis_process
# This command executes a model named "Latest_Fires" and exports the map layout as a PNG
command = [
    r'C:\Program Files\QGIS 3.40.7\bin\qgis_process-qgis-ltr.bat',  # Path to QGIS batch runner
    'run',                                      # Action: run model
    'model:Latest_Fires',                       # Model name (must be installed in QGIS)
    
    # --- Parameters for the model ---
    '--PROJECT_PATH=E:/QGIS PROJECTS/FIRMS/Final/Latest_Fires_Greece.qgz',  # QGIS project path
    '--distance_units=meters',                # Distance unit
    '--area_units=m2',                        # Area unit
    '--ellipsoid=EPSG:7030',                  # Ellipsoid used for geodesy calculations
    '--base_layer=E:/QGIS PROJECTS/FIRMS/Final/Daily_Fires.gpkg|layername=greece_map',  # Base map
    '--latest_fires=Latest_Fires',            # Output vector layer
    '--greece_fires=TEMPORARY_OUTPUT',        # Intermediate output layer
    '--eqgis_projectsfirmsdownloadlatest_fireszip=E:/QGIS PROJECTS/FIRMS/Download/latest_fires.zip',  # Zipped output
    '--daily_firespng=E:/QGIS PROJECTS/FIRMS/Final/Latest_Fires.png'  # 📸 Path to exported PNG map
]

print("🚀 Running QGIS model...")
# Run the command and halt if it fails
subprocess.run(command, check=True)
print("✅ Model completed, image exported.")

🚀 Running QGIS model...
✅ Model completed, image exported.


Upload image and delete file

In [4]:
# 📤 Step 2: Upload the exported PNG image and clean up
image_path = r'E:/QGIS PROJECTS/FIRMS/Final/Latest_Fires.png'

try:
    # Upload to imgbb and get the public URL
    url = upload_to_imgbb(image_path, title='Latest Fires Map')

    # ✅ Delete the image file after successful upload
    os.remove(image_path)
    print("🗑️ Image deleted locally.")

except Exception as e:
    # ⚠️ Handle upload or file errors
    print("❌ Something went wrong:", e)

✅ Uploaded: https://i.ibb.co/XrJdf2nv/Latest-Fires-Map.png
🗑️ Image deleted locally.


Add script to task scheduler (Powershell)

In [ ]:
$Action = New-ScheduledTaskAction -Execute "python.exe" -Argument "C:\Path\To\myscript.py"
$Trigger = New-ScheduledTaskTrigger -Daily -At 12:00PM
Register-ScheduledTask -TaskName "RunMyPythonScriptDaily" -Action $Action -Trigger $Trigger -Description "Runs my Python script daily at noon" -User "$env:USERNAME" -RunLevel Highest -Force